In [1]:
import os
import pytorch_lightning as pl
import pandas as pd

from rdkit import Chem # safe import before ccdc imports
from torch_geometric.loader import DataLoader
from torch.utils.data import ConcatDataset

from conf_ensemble_dataset_in_memory import ConfEnsembleDataset
from litschnet import LitSchNet
from molsize_model import MolSizeModel
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm import tqdm

In [2]:
pl.seed_everything(42, workers=True)

Global seed set to 42


42

# Data preparation

In [3]:
# run once to preprocess datasets and generate chunks
dataset = ConfEnsembleDataset()
# dataset = ConfEnsembleDataset(dataset='platinum')

In [4]:
splits = ['random', 'scaffold', 'protein']

In [5]:
def get_loaders(split, 
                iteration, 
                data_dir='data/',
                ) :
    
    pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
    pdbbind_n_chunks = len(pdbbind_chunks)
    
    train_datasets = []
    val_datasets = []
    test_datasets = []
    
    if split in ['random', 'scaffold'] :
        
        with open(os.path.join(data_dir, f'ligand_{split}_splits', f'train_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            train_smiles = f.readlines()
            train_smiles = [smiles.strip() for smiles in train_smiles]

        with open(os.path.join(data_dir, f'ligand_{split}_splits', f'val_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            val_smiles = f.readlines()
            val_smiles = [smiles.strip() for smiles in val_smiles]

        with open(os.path.join(data_dir, f'ligand_{split}_splits', f'test_smiles_{split}_split_{iteration}.txt'), 'r') as f :
            test_smiles = f.readlines()
            test_smiles = [smiles.strip() for smiles in test_smiles]

        for chunk_number in tqdm(range(pdbbind_n_chunks)) :

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          smiles_list=train_smiles)
            train_datasets.append(dataset)

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          smiles_list=val_smiles)
            val_datasets.append(dataset)

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          smiles_list=test_smiles)
            test_datasets.append(dataset)
            
    else : #protein split
        
        with open(os.path.join(data_dir, 'protein_similarity_splits', f'train_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
            train_pdbs = f.readlines()
            train_pdbs = [pdb.strip() for pdb in train_pdbs]

        with open(os.path.join(data_dir, 'protein_similarity_splits', f'val_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
            val_pdbs = f.readlines()
            val_pdbs = [pdb.strip() for pdb in val_pdbs]

        with open(os.path.join(data_dir, 'protein_similarity_splits', f'test_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
            test_pdbs = f.readlines()
            test_pdbs = [pdb.strip() for pdb in test_pdbs]

        for chunk_number in tqdm(range(pdbbind_n_chunks)) :

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          pdb_ids_list=train_pdbs)
            train_datasets.append(dataset)

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          pdb_ids_list=val_pdbs)
            val_datasets.append(dataset)

            dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                          pdb_ids_list=test_pdbs)
            test_datasets.append(dataset)

    train_dataset = ConcatDataset(train_datasets)
    val_dataset = ConcatDataset(val_datasets)
    test_dataset = ConcatDataset(test_datasets)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)
    
    return train_loader, val_loader, test_loader

In [6]:
#for split in splits :
for split in ['protein'] :
    
    for iteration in range(5) :
    
        train_loader, val_loader, test_loader = get_loaders(split, iteration)
        
        experiment_name = f'{split}_split_{iteration}'
        if not experiment_name in os.listdir('lightning_logs') :
            litschnet = LitSchNet()
            logger = TensorBoardLogger(save_dir=os.getcwd(), version=experiment_name, name="lightning_logs")
            trainer = pl.Trainer(logger=logger, callbacks=[EarlyStopping(monitor="val_loss", patience=5)], gpus=1)
            trainer.fit(litschnet, train_loader, val_loader)
            trainer.test(litschnet, test_loader)
            
        experiment_name = f'{split}_split_{iteration}_molsize'
        if not experiment_name in os.listdir('lightning_logs') :
            molsize_model = MolSizeModel()
            logger = TensorBoardLogger(save_dir=os.getcwd(), version=experiment_name, name="lightning_logs")
            trainer = pl.Trainer(logger=logger, callbacks=[EarlyStopping(monitor="val_loss", patience=5)], gpus=1)
            trainer.fit(molsize_model, train_loader, val_loader)
            trainer.test(molsize_model, test_loader)

100%|████████████████████████████████████████████| 3/3 [07:54<00:00, 158.33s/it]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | schnet     | SchNet    | 455 K 
1 | leaky_relu | LeakyReLU | 0     
2 | sigmoid    | Sigmoid   | 0     
-----------------------------------------
455 K     Trainable params
0         Non-trainable params
455 K     Total params
1.823     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:57: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.5530821084976196}
--------------------------------------------------------------------------------


In [7]:
from collections import defaultdict
n_bioactive_conformations = defaultdict(list)
n_conformations = defaultdict(list)
data_dir = 'data/'

for split in splits :
    
    for iteration in range(5) :
    
        pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
        pdbbind_n_chunks = len(pdbbind_chunks)

        train_datasets = []
        val_datasets = []
        test_datasets = []

        if split in ['random', 'scaffold'] :

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'train_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                train_smiles = f.readlines()
                train_smiles = [smiles.strip() for smiles in train_smiles]

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'val_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                val_smiles = f.readlines()
                val_smiles = [smiles.strip() for smiles in val_smiles]

            with open(os.path.join(data_dir, f'ligand_{split}_splits', f'test_smiles_{split}_split_{iteration}.txt'), 'r') as f :
                test_smiles = f.readlines()
                test_smiles = [smiles.strip() for smiles in test_smiles]

            for chunk_number in tqdm(range(pdbbind_n_chunks)) :

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=train_smiles)
                train_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=val_smiles)
                val_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              smiles_list=test_smiles)
                test_datasets.append(dataset)

        else : #protein split

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'train_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                train_pdbs = f.readlines()
                train_pdbs = [pdb.strip() for pdb in train_pdbs]

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'val_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                val_pdbs = f.readlines()
                val_pdbs = [pdb.strip() for pdb in val_pdbs]

            with open(os.path.join(data_dir, 'protein_similarity_splits', f'test_pdb_protein_similarity_split_{iteration}.txt'), 'r') as f :
                test_pdbs = f.readlines()
                test_pdbs = [pdb.strip() for pdb in test_pdbs]

            for chunk_number in tqdm(range(pdbbind_n_chunks)) :

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=train_pdbs)
                train_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=val_pdbs)
                val_datasets.append(dataset)

                dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                              pdb_ids_list=test_pdbs)
                test_datasets.append(dataset)

        train_dataset = ConcatDataset(train_datasets)
        val_dataset = ConcatDataset(val_datasets)
        test_dataset = ConcatDataset(test_datasets)
        
        d = {
            'train' : train_dataset,
            'val' : val_dataset,
            'test' : test_dataset
        }
        for s, dataset in d.items() :
            n_conformations[s].append(len(dataset))
            n_bio = 0
            for data in dataset :
                if data.rmsd == 0 :
                    n_bio = n_bio + 1
            n_bioactive_conformations[s].append(n_bio)

100%|████████████████████████████████████████████| 3/3 [08:05<00:00, 161.84s/it]


In [8]:
n_conformations

defaultdict(list,
            {'train': [777031,
              776369,
              780913,
              780115,
              778457,
              762491,
              765350,
              762653,
              758790,
              764836,
              767698,
              765616,
              773956,
              771798,
              774111],
             'val': [110778,
              110687,
              109384,
              109330,
              109139,
              92999,
              89046,
              96698,
              96347,
              87620,
              111143,
              104200,
              102109,
              104759,
              102038],
             'test': [108396,
              110217,
              108289,
              109775,
              108131,
              96219,
              97313,
              92358,
              96572,
              99253,
              101283,
              108352,
              102417,
              105115

In [9]:
n_bioactive_conformations

defaultdict(list,
            {'train': [13133,
              13137,
              13170,
              13177,
              13148,
              12377,
              12284,
              12570,
              12445,
              12457,
              12341,
              12307,
              12459,
              12456,
              12399],
             'val': [2601,
              2589,
              2525,
              2624,
              2644,
              1570,
              1622,
              1588,
              1549,
              1588,
              1582,
              1618,
              1536,
              1493,
              1597],
             'test': [2506,
              2546,
              2587,
              2618,
              2581,
              1550,
              1591,
              1339,
              1503,
              1452,
              1610,
              1600,
              1532,
              1582,
              1533]})